In [1]:
####### dropout resnet18 vs without dropout
#### 
import torch
import sys
sys.path.append("..")
import numpy as np

CLEAN_PATH = "/home/yifan/dataset/resnet18_with_dropout/pairflip/cifar10/0/"
REF_PATH = "/home/yifan/dataset/clean/pairflip/cifar10/0"


ENCODER_DIMS=[512,256,256,256,256,2]
DECODER_DIMS= [2,256,256,256,256,512]
VIS_MODEL_NAME = 'vis'

DEVICE='cuda:0'
########## initulize reference data and target data
from alignment.data_preprocess import DataInit
REF_EPOCH = 200
TAR_EPOCH = 200
ref_datainit = DataInit(REF_PATH,REF_PATH,REF_EPOCH)
tar_datainit = DataInit(CLEAN_PATH,CLEAN_PATH,TAR_EPOCH)
ref_model, ref_provider, ref_train_data, ref_prediction, ref_prediction_res, ref_scores = ref_datainit.getData()
tar_model, tar_provider, tar_train_data, tar_prediction, tar_prediction_res, tar_scores = tar_datainit.getData()


from alignment.ReferenceGenerator import ReferenceGenerator
gen = ReferenceGenerator(ref_provider=ref_provider, tar_provider=tar_provider,REF_EPOCH=REF_EPOCH,TAR_EPOCH=TAR_EPOCH,ref_model=ref_model,tar_model=tar_model,DEVICE=DEVICE)

absolute_alignment_indicates,predict_label_diff_indicates,predict_confidence_Diff_indicates,high_distance_indicates = gen.subsetClassify(mes_val_for_diff=18,mes_val_for_same=0.8,conf_val_for_diff=0.3,conf_val_for_same=0.05)


from representationTrans.trans_visualizer_border import visualizer
from singleVis.SingleVisualizationModel import VisModel
from singleVis.projector import TimeVisProjector
model = VisModel(ENCODER_DIMS, DECODER_DIMS)

I = np.eye(512)
projector = TimeVisProjector(vis_model=model, content_path=REF_PATH, vis_model_name=VIS_MODEL_NAME, device="cpu")
vis = visualizer(ref_provider, I,I, np.dot(ref_provider.train_representation(TAR_EPOCH),I), projector, 200,[0,1],'tab10')


/home/yifan/miniconda3/envs/deepdebugger/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NET resnet18
Finish initialization...


100%|██████████| 250/250 [00:00<00:00, 480.86it/s]


NET resnet18_with_dropout
Finish initialization...


100%|██████████| 250/250 [00:00<00:00, 7875.68it/s]


absolute alignment indicates number: 106 label diff indicates number: 12 confidence diff indicates number: 16 high distance number: 97


In [9]:
X = ref_train_data
Y = tar_train_data

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn import datasets

def gram_linear(x):
    """Compute Gram (kernel) matrix for a linear kernel."""
    return torch.matmul(x, x.t())

def center_gram(K):
    """Center a symmetric Gram matrix."""
    n = K.shape[0]
    unit = torch.ones([n, n], device=K.device) / n
    return K - torch.matmul(K, unit) - torch.matmul(unit, K) + torch.matmul(torch.matmul(unit, K), unit)

def cka(X, Y):
    """Compute CKA between two matrices."""
    X_centered = center_gram(gram_linear(X))
    Y_centered = center_gram(gram_linear(Y))
    X_normalized = torch.linalg.inv(torch.sqrt(torch.diag(torch.diag(X_centered))))
    Y_normalized = torch.linalg.inv(torch.sqrt(torch.diag(torch.diag(Y_centered))))
    X_normalized_centered = torch.matmul(torch.matmul(X_normalized, X_centered), X_normalized)
    Y_normalized_centered = torch.matmul(torch.matmul(Y_normalized, Y_centered), Y_normalized)
    cka = torch.trace(torch.matmul(X_normalized_centered, Y_normalized_centered)) / torch.sqrt(torch.trace(torch.matmul(X_normalized_centered, X_normalized_centered)) * torch.trace(torch.matmul(Y_normalized_centered, Y_normalized_centered)))
    return cka.item()
# Define a neural network architecture
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 512)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Create a neural network and an optimizer
net = MyNet()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Train the neural network to minimize the CKA loss
for epoch in range(100):
    optimizer.zero_grad()
    X_trans = net(torch.from_numpy(X).float())
    Y_torch = torch.from_numpy(Y).float()
    cka_loss = 1 - cka(X_trans, Y_torch)
    mse_loss = torch.mean(torch.pow(X_trans - X, 2))
    loss = 0.5 * cka_loss + 0.5 + mse_loss
    loss.backward()
    optimizer.step()
    print("Epoch {}: loss={}".format(epoch+1, loss))

# Compute the CKA value between X_trans and Y
X_trans = net(torch.from_numpy(X).float())
Y_torch = torch.from_numpy(Y).float()
cka_value = cka(X_trans, Y_torch)
print("CKA value between X_trans and Y: {}".format(cka_value))